# <font color='HotPink'>**Data Mining Project 2025/2026 - Group 94** <br></font> Amazing International Airlines Inc.


**Diogo Henrique Lopes de Carvalho** ***(20221935)***

**Maria Luiza Salum Alves Corrêa** ***(20221902)***

**Ricardo Miguel Silva Martins Pereira** ***(20250343)***

*** 

## **Index**

***First Delivery***
1. [Business Understanding](#1)
1. [Imports](#2)
1. [Data Understanding](#3)
    1. [The Dataset](#3_1)
1. [Data Preparation](#4)
    1. [Data Cleaning](#4_1)
    1. [Variable Classification](#4_2)
    1. [Summary Statistics](#4_3)
    1. [Data Visualisation](#4_4)
    1. [Feature Engineering](#4_5)

***Second Delivery***

5. [Clustering](#5)
5. [Evaluation](#6)
5. [Suggestions](#7)

#
***

## 1. <a id="1">Business Understanding</a>

Our client, **Amazing International Airlines Inc.**, tasked our consulting team with the analysis of their customer loyalty membership data and corresponding flight activity collected over a three-year period. <br>

The goal of our analysis is to create useful insights through a segmentation process that consists of grouping and viewing the available data by different lenses; such as *economic contribution*, *behaviours in purchases and travels*, and *demographic divisions*. <br>

After sharing the insights, our team will evaluate the results and give AIAI suggestions on how to adapt their services, rewards and communications to meet the needs of the most relevant groups found.

***

## 2. <a id="2">Imports</a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from scipy import stats
from itertools import product


# import itertools

***

## 3. <a id="3">Data Understanding</a>

### 1. <a id="3_1">The Dataset</a>

The dataset given to our team consists of three `csv` files:

- `DM_AIAI_CustomerDB.csv`

*Three (3) years of data related to Customers in the Loyalty Program*
- `DM_AIAI_FlightsDB.csv`

*Three (3) years of data related to Flights of the Loyalty Program Members*
- `DM_AIAI_Metadata.csv`

*Description of the Variables in the CustomerDB and FlightDB Files*

In [ ]:
customerDB = pd.read_csv(r'data\DM_AIAI_CustomerDB.csv', sep=',')
flightDB = pd.read_csv(r'data\DM_AIAI_FlightsDB.csv', sep=',')
metadata = pd.read_csv(r'data\DM_AIAI_Metadata.csv', sep=';')

customerDB_copy = customerDB.copy()
flightDB_copy = flightDB.copy()

In [ ]:
pd.set_option('display.max_colwidth', None)  # To see full descriptions
metadata

#### Initial Conclusions
*What conclusions can we make from the metadata?*

##### Customer DB

**Useful:**
- `Loyalty#`: Will be used. Connects Customer's and Flights' information.

**Will be Dropped:**
- `First Name`, `Last Name`, `Customer Name`: First two are redundant since we have the full name, but this information is irrelevant for clustering anyways. We'll drop everything after data cleaning. *(Keeping `Customer Name` for now to check something with the IDs)*
- `Latitude`, `Longitude`, `Postal Code`: No need for specific customer location. *(Spoilers: We checked, some coordinates lead to nowhere and just make no sense overall)*

**No Conclusions:** <br>
*May be relevant. No conclusions about data quality from metadata.*
- `Country`
- `Province or State`
- `City`
- `Postal Code`
- `Gender`
- `Education`
- `Location Code`
- `Income`
- `Marital Status`
- `Loyalty Status`
- `EnrollmentDataOpening`
- `CancellationDate`
- `Customer Lifetime Value`
- `EnrollmentType`

In [ ]:
customerDB_copy.drop(columns=['First Name', 'Last Name', 'Latitude', 'Longitude', 'Postal code'], inplace=True)
customerDB_copy.info()

From the non-null count, we can see that there are only three variables with missing values: `Income`, `CancellationDate` and `Customer Lifetime Value`. Having null values in `CancellationDate` clearly makes sense as we don't expect all customers in the dataset to have cancelled their membership. For `Income` and `Customer Lifetime Value`, we'll try to understand the reason later.

In [ ]:
# There's a random column with no name, let's drop it
customerDB_copy.drop(columns=['Unnamed: 0'], inplace=True)

##### Flights DB

**Useful:**
- `Loyalty#`: Will be used. Connects Customer's and Flights' information.

**Will be Dropped:**
- `YearMonthDate`: Contains the same info as `Month` and `Year` with a constant date.
- `DollarCostPointsRedeemed`: Redundant. `PointsRedeemed` gives the same information.

**No Conclusions:** <br>
*May be relevant. No conclusions about data quality from metadata.*
- `Year`
- `Month`
- `NumFlights`
- `NumFlightsWithCompanions`*
- `DistanceKM`
- `PointsAccumulated`
- `PointsRedeemed`

**We will assume `NumFlightsWithCompanions` is also a "count by month" even though it's not mentioned in the metadata.*

In [ ]:
flightDB_copy.drop(columns=['YearMonthDate', 'DollarCostPointsRedeemed'], inplace=True)
flightDB_copy.info()

There are no null values in any variable. We'll take a closer look into the dataset when preparing the data. There might be missing values that are non-null since they could look like "-", "Null", "Missing", etc.

***

## 4. <a id="4">Data Preparation</a>

There's a possibility of having some rows that the only plausible option is to remove from the dataset due to uncorrectable errors. For this reason, we are going to define variables to keep track of how many rows we are dropping.

We are going to assume that an acceptable threshold is 5% of the dataset.

In [ ]:
customer_dropped_rows = 0
customer_original_rows = customerDB.shape[0]
customer_dropped_percentage = 0
customer_available_to_drop = (customer_original_rows * 0.05) - customer_dropped_rows
print(f"Available rows to drop in customerDB: {customer_available_to_drop} ({(customer_available_to_drop/customer_original_rows)*100:.0f}%)")

flight_dropped_rows = 0
flight_original_rows = flightDB.shape[0]
flight_dropped_percentage = 0
flight_available_to_drop = (flight_original_rows * 0.05) - flight_dropped_rows
print(f"Available rows to drop in flightDB: {flight_available_to_drop} ({(flight_available_to_drop/flight_original_rows)*100:.0f}%)")

In [ ]:
customerDB_copy.head()

In [ ]:
# Seeing the different values in each variable and if they're less than 15, printing them bellow
for variable in customerDB_copy.columns:
    print(f"{variable}: {customerDB_copy[variable].nunique()} unique values\n")
    if customerDB_copy[variable].nunique() < 15:
        print(f"{customerDB_copy[variable].value_counts()}\n")

In [ ]:
flightDB_copy.head()

In [ ]:
# Seeing the different values in each variable and if they're less than 15, printing them bellow
for variable in flightDB_copy.columns:
    print(f"{variable}: {flightDB_copy[variable].nunique()} unique values\n")
    if flightDB_copy[variable].nunique() < 15:
        print(f"{flightDB_copy[variable].value_counts()}\n")

### 1. <a id="4_1">Data Cleaning</a>

##### Loyalty# *(+ Customer Name)*

In [ ]:
# Confirming how many rows have repeated Loyalty#
print(customerDB_copy['Loyalty#'].duplicated().sum())
# Confirming if the whole row is duplicated
print(customerDB_copy.duplicated().sum())

We kept `Customer Name` before so we could check if they're the same for repeated Loyalty IDs, but this is not the case as the unique values count is 16921. This means that we cannot use the name to fix the issue with duplicated IDs (they're not the same person).

We could assign new random values to the repeated IDs, but the FlightsDB dataset is connected to each customer through this variable, so it makes no sense to change it in the CustomerDB since this would mean that they have no associated flights.

We'll now drop `Customer Name` and remove the rows with repeated Loyalty ID.

In [ ]:
# Dropping Customer Name
customerDB_copy.drop(columns=['Customer Name'], inplace=True)

In [ ]:
# Keeping only one of the duplicated rows and adding the number of dropped rows to the counter
for loyalty_id in customerDB_copy['Loyalty#'][customerDB_copy['Loyalty#'].duplicated()]: # for each duplicated Loyalty#
    duplicated_rows = customerDB_copy[customerDB_copy['Loyalty#'] == loyalty_id] # getting the duplicated rows
    customerDB_copy.drop(duplicated_rows.index[1:], inplace=True) # dropping from the second to the last (just in case there are more than 2)
    customer_dropped_rows += len(duplicated_rows) - 1 # adding the number of dropped rows to the counter
    
customer_dropped_percentage = customer_dropped_rows / customer_original_rows * 100
customer_available_to_drop -= customer_dropped_rows
print(f"Total dropped rows: {customer_dropped_rows}, which is {customer_dropped_percentage:.2f}% of the original dataset. We can still drop {customer_available_to_drop} rows.")

##### Country

`Country` is a constant feature (Canada). We'll drop this column.

In [ ]:
# Dropping Country 
customerDB_copy.drop(columns=['Country'], inplace=True)

#### City

Checking if all cities in the list are Canadian cities.

In [ ]:
customerDB_copy['City'].unique()

#### Income - Customer Lifetime Value

We'll see if the null values in `Income` are related to the null values in `Customer Lifetime Value`.

In [ ]:
income_null = customerDB_copy[customerDB_copy['Income'].isnull()]
clv_null = customerDB_copy[customerDB_copy['Customer Lifetime Value'].isnull()]
income_clv_null = customerDB_copy[(customerDB_copy['Income'].isnull()) & (customerDB_copy['Customer Lifetime Value'].isnull())]
print(f"Income null values: {len(income_null)}\nCLV null values: {len(clv_null)}\nBoth null values: {len(income_clv_null)}")

In [ ]:
# seeing these customers in customerDB_copy
customerDB_copy.loc[income_clv_null.index]

Now we know they're the same customers.

Also, we can see that each one of these customers have the same `EnrollmentDateOpening` and `CancellationDate`, which makes us believe that they will have no relevant information in the flight dataset. If they don't, we'll just drop the rows.

In [ ]:
# seeing if there are any Loyalty# in customerDB_copy that are not in flightDB_copy
missing_loyalty = customerDB_copy.index.difference(flightDB_copy['Loyalty#'].unique())
# seeing if they're the ones with null values in Customer Lifetime Value
clv_null_loyalty = customerDB_copy[customerDB_copy['Customer Lifetime Value'].isnull()].index
meets = []
for index in missing_loyalty:
    if index in clv_null_loyalty:
        meets.append(index)
print(f'Loyalty# missing in flightDB_copy and null CLV/Income: \n{meets}\nTotal: {len(meets)}')

As expected, they have no flight information at all! Let's drop them.

In [ ]:
customerDB_copy.drop(index=meets, inplace=True)
customer_dropped_rows += len(meets)
customer_dropped_percentage = customer_dropped_rows / customer_original_rows * 100
customer_available_to_drop -= len(meets)
print(f"Total dropped rows: {customer_dropped_rows}, which is {customer_dropped_percentage:.2f}% of the original dataset. We can still drop {customer_available_to_drop} rows.")

Let's see the values equal to zero...

In [ ]:
income_zero = customerDB_copy[customerDB_copy['Income'] == 0]
clv_zero = customerDB_copy[customerDB_copy['Customer Lifetime Value'] == 0]
income_clv_zero = customerDB_copy[(customerDB_copy['Income'] == 0) & (customerDB_copy['Customer Lifetime Value'] == 0)]
print(f"Income zero values: {len(income_zero)}\nCLV zero values: {len(clv_zero)}\nBoth zero values: {len(income_clv_zero)}\n")
print(income_zero['Customer Lifetime Value'])

We'll try to understand if there's a pattern for the 0€ income values in the Data Visualisation section. Maybe a specific type of customer decided to keep their income a secret..? We don't know yet, but we can see that these customers, even though they have no revealed `Income`, have an estimated CLV.

#### CancellationDate

Checking if there are dates in `CancellationDate` that are earlier than dates in `EnrollmentDateOpening`.

In [ ]:
customerDB_copy['EnrollmentDateOpening'] = pd.to_datetime(customerDB_copy['EnrollmentDateOpening'], errors='coerce')
customerDB_copy['CancellationDate'] = pd.to_datetime(customerDB_copy['CancellationDate'], errors='coerce')
invalid_dates = customerDB_copy[customerDB_copy['CancellationDate'] <= customerDB_copy['EnrollmentDateOpening']]
print(f"Number of rows with CancellationDate before EnrollmentDateOpening: {len(invalid_dates)}")
invalid_dates

In [ ]:
# Seeing the total flight dataset data per customer in the invalid_dates
invalid_loyalty_numbers = invalid_dates['Loyalty#'].values
invalid_flights = flightDB_copy[flightDB_copy['Loyalty#'].isin(invalid_loyalty_numbers)]
invalid_flights_grouped = invalid_flights.groupby('Loyalty#').agg({
    'NumFlights': 'sum',
    'NumFlightsWithCompanions': 'sum',
    'DistanceKM': 'sum',
    'PointsAccumulated': 'sum'
}).reset_index()
invalid_flights_grouped

Since we have relevant flight data for customers with invalid dates, we are going to assume that the customer had a previous membership, cancelled it, and then decided to become a member once again.

For this reason, we're just going to replace these dates in `CancellationDate` with Na.

*We're not adding this to the dropped rows count as we only did a replacement.*

In [ ]:
for row in invalid_dates:
    customerDB_copy.loc[invalid_dates.index, 'CancellationDate'] = pd.NaT

#### CustomerDB - FlightDB Relation

In [ ]:
flightDB_copy.shape[0]/customerDB_copy.shape[0]

Every customer in the loyalty program should have a row for each one of the 36 months in the dataset. The number above shows that there are more months registered than customers. We'll take a look into this.

In [ ]:
# for each loyalty number, counting how many rows there are in the flights dataset
flight_counts = flightDB_copy['Loyalty#'].value_counts()
loyalty_over_36 = flight_counts[flight_counts > 36]
print(f"Number of Loyalty# with more than 36 rows: {len(loyalty_over_36)}")

# for each loyalty number, counting how many months are missing in the flights dataset
months_per_loyalty = flightDB_copy.groupby('Loyalty#').size()
missing_months = months_per_loyalty[months_per_loyalty < 36]
print(f"Number of Loyalty# with less than 36 months: {len(missing_months)}")

In [ ]:
# showing the rows for one Loyalty# with more than 36 rows
flightDB_copy[flightDB_copy['Loyalty#'] == loyalty_over_36.index[0]]

For each repeated month, we'll keep the row with the less amount of null values.

In [ ]:
non_null_counts = flightDB_copy.notnull().sum(axis=1) # counting non-null values in each row
row_to_keep = (flightDB_copy.assign(non_null_count=non_null_counts).groupby(['Loyalty#', 'Year', 'Month'])['non_null_count'].idxmax()) # getting the index of the row with the most non-null values for each Loyalty#, Year, Month
flightDB_copy = flightDB_copy.loc[row_to_keep].copy() # keeping only the rows with the most non-null values

# updating the dropped rows counter
flight_dropped_rows = flight_original_rows - flightDB_copy.shape[0]
flight_available_to_drop -= flight_dropped_rows

print(f"Total dropped rows: {flight_dropped_rows}, which is {(flight_dropped_rows/flight_original_rows)*100:.2f}% of the original dataset. We can still drop {flight_available_to_drop} rows.")

*Side comment: the same percentage of rows was dropped when we were cleaning the duplicate customers... This could mean that the people dropped in this previous step had an already existing ID assigned to them when creating their account. This would have mixed their data with the other customer with the existing ID.*

*There's no way of identifying which flight information is related to each one of these customers, so we'll need to work with what we have.*

In [ ]:
# for each loyalty number, counting how many rows there are in the flights dataset
flight_counts = flightDB_copy['Loyalty#'].value_counts()
loyalty_over_36 = flight_counts[flight_counts > 36]
print(f"Number of Loyalty# with more than 36 rows: {len(loyalty_over_36)}")

# for each loyalty number, counting how many months are missing in the flights dataset
months_per_loyalty = flightDB_copy.groupby('Loyalty#').size()
missing_months = months_per_loyalty[months_per_loyalty < 36]
print(f"Number of Loyalty# with less than 36 months: {len(missing_months)}")

### 2. <a id="4_2">Variable Classification</a>

*It makes no sense to make `Loyalty#` the index in the flightDB dataset as the same customer may have multiple flights.*

In [ ]:
# Making Loyalty# the index for the customerDB
customerDB_copy.set_index('Loyalty#', inplace=True)

We're going to categorise the different variables in both datasets.


For `customerDB`, we'll have Nominal, Ordinal, Numerical, and DateTime variables. We're diving the categorical variables in Nominal and Ordinal because the latter will have its variables used for the Encoded category.

For `flightDB`, we'll only have Numerical and DateTime variables since this dataset has no categorical variables.

In [ ]:
customerDB_copy.info()

In [ ]:
customer_nominal_variables = ['Province or State', 'City', 'Gender', 'Location Code', 'Marital Status', 'EnrollmentType']
customer_ordinal_variables = ['Education', 'LoyaltyStatus']
customer_numerical_variables = customerDB_copy.select_dtypes(include=[np.number]).columns.tolist()
customer_datetime_variables = customerDB_copy.select_dtypes(include=['datetime']).columns.tolist()

In [ ]:
# Encoding ordinal variables
education_map = {'Bachelor':3, 'College':2, 'High School or Below':1, 'Doctor':5, 'Master':4}
loyaltystatus_map = {'Star':1, 'Nova':2, 'Aurora':3}
for variable in customer_ordinal_variables:
    if variable == 'Education':
        customerDB_copy['Education Encoded'] = customerDB_copy[variable].map(education_map)
    elif variable == 'LoyaltyStatus':
        customerDB_copy['LoyaltyStatus Encoded'] = customerDB_copy[variable].map(loyaltystatus_map)
customer_ordinal_variables_encoded = ['Education Encoded', 'LoyaltyStatus Encoded'] # New list of encoded ordinal variables

In [ ]:
flightDB_copy.info()

In [ ]:
flight_numerical_variables = flightDB_copy.select_dtypes(include=['float']).columns.tolist()
flight_date_variables = ['Year', 'Month']

*We could change `Year` and `Month` into a datetime column, but we figured they would be more useful for analysis as separate variables.*

### 3. <a id="4_3">Summary Statistics</a>

In [ ]:
customerDB_copy[customer_ordinal_variables].describe().T

In [ ]:
customerDB_copy[customer_numerical_variables].describe().T

`Customer Lifetime Value` seems to have outliers on the right, as the mean is higher than the median. This means that *some* customers have a high Lifetime Value, but the majority doesn't. It makes sense from a logical point of view as we don't see most people having frequent flights. However, we'll try to understand how this variable relates to the rest in the Data Visualisation section.

As said previously, in clustering, we will try to understand if there's a pattern in the missing values in `Income`. *(i.e. If they're missing completely at random, at random or not at random)*

In [ ]:
flightDB_copy[flight_numerical_variables].describe().T

In the Flights dataset, we can see that most variables have many rows with null values.

We can conclude that 25% of the registered *months by customer* have no flights. This could mean that some months have few or no flights or that some customers have no flights in most or all months.

The `NumFlightsWithCompanions` variable is clearly related to the `NumFlights`, as one indicates the number of flights *with companions* and the other the number of flights the customer had in the month in total, respectively, so it makes sense that it has even more null values (at least 50%).

The remaining values follow the same logic. They're all related to the total number of flights, so it makes sense that at least 25% of their values are null since 25% of `NumFlights`'s values is also null.

`PointsRedeemed` and `DollarCostPointsRedeemed` also show that most customers (75% or more) don't redeem their available points.

Overall, we can conclude that every numerical variable of this dataset is left-skewed as the mean is much higher than the median in every case.

### 4. <a id="4_4">Data Visualisation</a>

In this section, we’ll visually explore the data to understand the distributions of each variable and the relationships between them. We’ll also use this step to confirm and inspect any outliers identified during the summary statistics stage.

Once we’ve gained a solid understanding of the existing variables, we’ll move on to the feature engineering phase, where we’ll create new features and aggregations based on these insights. Afterwards, we’ll do a brief round of statistical and visual checks to ensure the new variables behave as expected and add meaningful value to the dataset.

#### Numerical Data

#### Categorical Data

#### Correlations

##### Numerical & Numerical Data

##### Categorical & Categorical Data

##### Numerical & Categorical Data

### 5. <a id="4_5">Feature Engineering</a>

To use data on the customer level, we'll have to calculate the total and average values on the flight dataset since we currently have 36 rows (12 months for 3 years) for each customer.

In [ ]:
# aggregate all flight metrics at once and merge in a single operation
agg = (flightDB_copy.groupby('Loyalty#')[flight_numerical_variables].agg(['sum', 'mean']))

new_cols = []
for col, stat in agg.columns:
    if stat == 'sum':
        prefix = 'Total'
    else:
        prefix = 'Average'
    new_cols.append(f"{prefix} {col}")
agg.columns = new_cols

customerDB_copy = customerDB_copy.merge(agg, left_index=True, right_index=True, how='left')

customerDB_copy.info()

#
***

## 5. <a id="5">Clustering</a>

***

## 6. <a id="6">Evaluation</a>

***

## 7. <a id="7">Suggestions</a>

***